# P-P Investing
Jianlei 
See My App: http://p2p-invest.herokuapp.com
Many people have money sitting in a bank account seek ways to invest. Comparing to the rather unstable stock market, Peer to Peer(P2P) would be a great choice. P2P lending is the method of lending money to individuals or businesses through online services. On the investor's side, the investment return is much higher than bank CD account, and safer than stock market.

As an investor, there are two things I care about, first when I lend money out, I defiantly want it back, so the charge-off rate would be important. Secondly, I would want to earn a good interest. As you may know, risker loans come with higher interest rate, to find a happy balance, I have created this app that will recommend safer loans given a given risk appetite. 

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import time
from datetime import date
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, auc, confusion_matrix
from datetime import timedelta, datetime
from sklearn.ensemble import GradientBoostingClassifier
% matplotlib inline
# Always make it pretty.
plt.style.use('ggplot')
import math
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from urllib.request import urlopen, Request
import requests
from sklearn.externals import joblib as pickle
from sklearn import base

In [2]:
from transformers import ColumnSelectTransformer, CreditTimeTransformer, NaInputeTransformer, EstimatorTransformer, ColumnUnSelectTransformer,OneColumnSelectTransformer,ReshapeTransformer,MyLabelEncoder


# Load Lending Club History Data 

In [3]:
Lending_club = pd.read_csv('Lending_club_train.csv')

/opt/conda/envs/data3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,48,128,129,130,133,134,135,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
emplength_dict = {'1':12, '10+ years':120, '2 years':24, '3 years':36, '< 1 year':6, '5 years':60,
                  '1 year':12, '7 years': 84, '6 years':72, '4 years':48, '8 years':96, 
                  '9 years':108, 0 : 0}
Lending_club.emplength.fillna(0, inplace = True)
Lending_club['emplength'] = Lending_club['emplength'].apply(lambda x: emplength_dict[x])

In [5]:
LendingA = Lending_club[Lending_club['grade'] == 'A']
LendingB = Lending_club[Lending_club['grade'] == 'B']
LendingC = Lending_club[Lending_club['grade'] == 'C']
LendingD = Lending_club[Lending_club['grade'] == 'D']
LendingE = Lending_club[Lending_club['grade'] == 'E']
LendingF = Lending_club[Lending_club['grade'] == 'F']
LendingG = Lending_club[Lending_club['grade'] == 'G']
yA = LendingA.loanstatus
yB = LendingB.loanstatus
yC = LendingC.loanstatus
yD = LendingD.loanstatus
yE = LendingE.loanstatus
yF = LendingF.loanstatus
yG = LendingG.loanstatus

In [6]:
agree = pickle.load('agree.pickle')

# Get current Listing From Lending Club API

In [7]:
headers = {'Authorization' : 'p96JwpSHaxLH5U4PFPwcYWv9ZhY='} # this is my token, you should put in yours and set it as a system variable to invest
result = requests.get('https://api.lendingclub.com/api/investor/v1/loans/listing', headers = headers)
Listing = result.json()
Listing = pd.DataFrame(Listing['loans'])

In [8]:
Listing['issued'] = date.today()
Listing = Listing[Listing['term'] == 36] # I only look at 36 month loans

# Data Processing

In [9]:
List_col = [i.lower() for i in Listing.columns.values]
Listing.columns = List_col #normalize the formmate issue with the data set

In [10]:
Listing = Listing[agree]
ListingA = Listing[Listing['grade'] == 'A']
ListingB = Listing[Listing['grade'] == 'B']
ListingC = Listing[Listing['grade'] == 'C']
ListingD = Listing[Listing['grade'] == 'D']
ListingE = Listing[Listing['grade'] == 'E']
ListingF = Listing[Listing['grade'] == 'F']
ListingG = Listing[Listing['grade'] == 'G']

## Pick out the relevent columns

Pickout the columns to keep, first the share variable denote all the columns that exists both on my historical and the listing set. this will make sure there is no information leakage.

In [11]:
columns_keep = ['addrstate','annualinc','chargeoffwithin12mths','delinq2yrs', 'purpose',
                'delinqamnt', 'dti','earliestcrline','emplength','ficorangehigh','pubrec',
               'installment','intrate','loanamount', 'numoprevtl', 'numrevaccts',
       'homeownership','numbcsats','revolbal','revolutil','taxliens','totcurbal'
               ]

A not about missing value inputation, there are three ways of inputing missing values, the zero inputation, mean, and major class.

In [12]:
zerox_inp = ['chargeoffwithin12mths','delinq2yrs', 'delinqamnt','emplength',
             'numoprevtl', 'numrevaccts','numbcsats','pubrec','revolbal',
             'revolutil','taxliens','totcurbal'
            ]
mean_inp = ['ficorangehigh','annualinc','dti']
maj_inp = ['addrstate','purpose', 'homeownership', 'earliestcrline']

The CreditTimeTransformer claculate the credit length from the loan application date. since the data we have is only the earlist credit line, but it will be more meaninful to compare the credit length at the time of the loan

# Pipeline
## Data Cleaning

In [13]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('ctt', CreditTimeTransformer('earliestcrline', 'issued')),
    ('cst', ColumnSelectTransformer(columns_keep)),
    ('nit', NaInputeTransformer(zerox_inp, mean_inp, maj_inp))
])

## Puspose pipe

In [14]:
purpose_pipe = Pipeline([('cst1', OneColumnSelectTransformer('purpose')),
    ('lab', MyLabelEncoder()),
    ('rft', ReshapeTransformer()),                     
    ('ohe', OneHotEncoder(sparse=False))
                        ])

## Home Ownership pipe

In [15]:
homeownership_pipe = Pipeline([('cst1', OneColumnSelectTransformer('homeownership')),
    ('lab', MyLabelEncoder()),
    ('rft', ReshapeTransformer()),                     
    ('ohe', OneHotEncoder(sparse=False))
                        ])

## Feature Union

In [16]:
from sklearn.pipeline import FeatureUnion
cata = ['addrstate', 'purpose', 'homeownership']
union =  FeatureUnion([
    ('cust', ColumnUnSelectTransformer(cata)),
    ('purpose_pipe', purpose_pipe),
    ('homeownership_pipe', homeownership_pipe)
    ])

## Full pipe

In [53]:
from sklearn.model_selection import GridSearchCV
param = {
    'C':[0.001,0.01,0.1,1,10,100],
    'penalty': ['l2', 'l1'],
    'class_weight': ['balanced']
}

full_pipe = Pipeline([
    ('pipe_clean', pipe),
    ('pipe_enc', union),
    ('clf', GridSearchCV(LogisticRegression(), param, cv = 3))
])

## pipeA

In [18]:
X_train, X_test, y_train, y_test = train_test_split(LendingA, yA, test_size=0.2, random_state=43)

In [21]:
pipeA = full_pipe.fit(X_train,y_train);

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [22]:
y_pred = full_pipe.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [25]:
MatrixA = confusion_matrix(y_test, y_pred)
NagitiveA = MatrixA[0][0]/(MatrixA[0][0]+MatrixA[1][0])*100
PositiveA = MatrixA[0][1]/(MatrixA[0][1]+MatrixA[1][1])*100
AllA = 100-sum(yA)/len(yA)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For A grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveA, AllA, (AllA - PositiveA)*100/AllA))

For A grade The Risk of using my model is: 3.2%, comparing to auto investment: 5.52% you are 42.0% safer


## pipeB

In [26]:
X_train, X_test, y_train, y_test = train_test_split(LendingB, yB, test_size=0.2, random_state=43)

In [ ]:
pipeB = full_pipe.fit(X_train,y_train);

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [ ]:
y_pred = pipeB.predict(X_test)

In [32]:
MatrixB = confusion_matrix(y_test, y_pred)
NagitiveB = MatrixB[0][0]/(MatrixB[0][0]+MatrixB[1][0])*100
PositiveB = MatrixB[0][1]/(MatrixB[0][1]+MatrixB[1][1])*100
AllB = 100-sum(yB)/len(yB)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For B grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveB, AllB, (AllB - PositiveB)*100/AllB))

For B grade The Risk of using my model is: 6.86%, comparing to auto investment: 10.7% you are 35.7% safer


In [36]:
pipeA.named_steps['clf'].best_params_

{'C': 10, 'class_weight': 'balanced', 'penalty': 'l1'}

In [37]:
# since we have already know the best params 
param = {
    'C':[0.1,1,10],
    'penalty': [ 'l1'],
    'class_weight': ['balanced']
}

full_pipe = Pipeline([
    ('pipe_clean', pipe),
    ('pipe_enc', union),
    ('clf', GridSearchCV(LogisticRegression(), param, cv = 3))
])



## pipeC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(LendingC, yC, test_size=0.2, random_state=43)
pipeC = full_pipe.fit(X_train,y_train)
y_pred = pipeC.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [ ]:
MatrixC = confusion_matrix(y_test, y_pred)
NagitiveC = MatrixC[0][0]/(MatrixC[0][0]+MatrixC[1][0])*100
PositiveC = MatrixC[0][1]/(MatrixC[0][1]+MatrixC[1][1])*100
AllC = 100-sum(yC)/len(yC)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For C grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveC, AllC, (AllC - PositiveC)*100/AllC))

For C grade The Risk of using my model is: 14.0%, comparing to auto investment: 16.2% you are 13.8% safer


## pipeD

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(LendingD, yD, test_size=0.2, random_state=43)
pipeD = full_pipe.fit(X_train,y_train)
y_pred = pipeD.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [ ]:
MatrixD = confusion_matrix(y_test, y_pred)
NagitiveD = MatrixD[0][0]/(MatrixD[0][0]+MatrixD[1][0])*100
PositiveD = MatrixD[0][1]/(MatrixD[0][1]+MatrixD[1][1])*100
AllD = 100-sum(yD)/len(yD)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For D grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveD, AllD, (AllD - PositiveD)*100/AllD))

For D grade The Risk of using my model is: 17.3%, comparing to auto investment: 20.6% you are 16.0% safer


## pipeE

In [42]:
X_train, X_test, y_train, y_test = train_test_split(LendingE, yE, test_size=0.2, random_state=43)
pipeE = full_pipe.fit(X_train,y_train)
y_pred = pipeE.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [43]:
MatrixE = confusion_matrix(y_test, y_pred)
NagitiveE = MatrixE[0][0]/(MatrixE[0][0]+MatrixE[1][0])*100
PositiveE = MatrixE[0][1]/(MatrixE[0][1]+MatrixE[1][1])*100
AllE = 100-sum(yE)/len(yE)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For E grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveE, AllE, (AllE - PositiveE)*100/AllE))

For E grade The Risk of using my model is: 20.2%, comparing to auto investment: 23.9% you are 15.3% safer


## pipeF

In [44]:
X_train, X_test, y_train, y_test = train_test_split(LendingF, yF, test_size=0.2, random_state=43)
pipeF = full_pipe.fit(X_train,y_train)
y_pred = pipeF.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [45]:
MatrixF = confusion_matrix(y_test, y_pred)
NagitiveF = MatrixF[0][0]/(MatrixF[0][0]+MatrixF[1][0])*100
PositiveF = MatrixF[0][1]/(MatrixF[0][1]+MatrixF[1][1])*100
AllF = 100-sum(yF)/len(yF)*100
#print(Matrix, Nagitive*100, Positive*100, All*100)
print('For F grade The Risk of using my model is: {0:.3}%, comparing to auto investment: {1:.3}% you are {2:.3}% safer'.format( PositiveF, AllF, (AllF - PositiveF)*100/AllF))

For F grade The Risk of using my model is: 25.4%, comparing to auto investment: 28.3% you are 10.4% safer


## pipeG

In [60]:
X_train, X_test, y_train, y_test = train_test_split(LendingG, yG, test_size=0.2, random_state=42)
pipeG = full_pipe.fit(X_train,y_train)
y_pred = pipeG.predict(X_test)

/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/data3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

Now we have all the models for all risk appitite, this app will predict the relevently safer loan within each risk appitite. 

# Survival Feature
The next question asked is, what if I have already start a loan, how do I evaluate how safe it is. Here, I used the kaplan -merier survival analysis to estimate the survival probability given its current performance.


In [2]:
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()

In [ ]:
Lending_club = pd.read_csv('accepted_2007_to_2017Q3.csv')

/opt/conda/envs/data3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,18,48,58,117,128,129,130,133,134,135,138,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data Cleaning

In [ ]:
def Clean_sub_Lending_club(Lending_club):
    Lending_club_todrop = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv',
                      'verification_status','desc', 'title', 'zip_code','delinq_2yrs',
                      'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 
                      'revol_bal', 'revol_util', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                      'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount',
                        'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 
                       'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 
                       'hardship_last_payment_amount', 'disbursement_method', 'debt_settlement_flag', 'debt_settlement_flag_date', 
                       'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term',
                      'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
                       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 
                       'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 
                       'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 
                       'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 
                       'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 
                       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'revol_bal_joint', 
                       'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 
                       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 
                       'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog',
                        ]
    Lending_club.drop(Lending_club_todrop,axis = 1 ,inplace = True)
    Lending_club['earliest_cr_line'] = pd.to_datetime(Lending_club['earliest_cr_line'])
    Lending_club['issue_d'] = pd.to_datetime(Lending_club['issue_d'])
    Lending_club['last_pymnt_d'] = pd.to_datetime(Lending_club['last_pymnt_d'])
    Lending_club['next_pymnt_d'] = pd.to_datetime(Lending_club['next_pymnt_d'])
    Lending_club['last_credit_pull_d'] = pd.to_datetime(Lending_club['last_credit_pull_d'])
    Lending_club['dti_new'] = np.where(Lending_club['application_type']=='JOINT', Lending_club['dti_joint'],Lending_club['dti'])
    Lending_club['annual_inc'] = np.where(Lending_club['application_type']=='JOINT', Lending_club['annual_inc_joint'],Lending_club['annual_inc'])
    Lending_club_sub_col = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'tot_cur_bal',
                       'emp_length', 'home_ownership', 'annual_inc', 'issue_d', 
                        'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'dti', 
                        'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'pub_rec', 
                        'total_acc', 'initial_list_status' ,'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d','duration']
    Lending_club['last_pymnt_d'] = pd.to_datetime(Lending_club['last_pymnt_d'])
    Lending_club['next_pymnt_d'] = pd.to_datetime(Lending_club['next_pymnt_d'])
    Lending_club['duration'] = Lending_club['last_pymnt_d']-Lending_club['issue_d']
    Lending_club_sub = Lending_club[Lending_club_sub_col]
    Lending_club_sub['Credit_len'] =  Lending_club_sub['issue_d']- Lending_club_sub['earliest_cr_line']
    Lending_club_sub['Credit_day'] = Lending_club_sub['Credit_len'].apply(lambda x: x.days)
    Lending_club_sub['log_tot_Cur_bal'] = Lending_club_sub['tot_cur_bal'].apply([lambda x: math.log(x+1)])
    Lending_club_sub['log_income'] = Lending_club_sub['annual_inc'].apply(lambda x: math.log(x+1))
    Lending_club_sub_col = Lending_club_sub.columns.values
    Lending_club_na = Lending_club_sub.isnull().sum()/len(Lending_club_sub)
    Lending_club_sub.loc[(Lending_club_sub['home_ownership'] == 'OWN') | (Lending_club_sub['home_ownership'] == 'MORTGAGE') , 'home_ownership'] = 1
    Lending_club_sub.loc[(Lending_club_sub['home_ownership'] != 1)  , 'home_ownership'] = 0 
    Lending_club_sub.loc[(Lending_club_sub['loan_status'] == 'Current') | (Lending_club_sub['loan_status'] == 'Fully Paid') , 'loan_status'] = 'Good' 
    Lending_club_sub.loc[(Lending_club_sub['loan_status'] != 'Good')  , 'loan_status'] = 'Bad' 
    for i in range(len(Lending_club_na)):
        if Lending_club_na[i] > 0.8:
            Lending_club_sub.drop(Lending_club_col[i],axis = 1, inplace = True)
    for i in Lending_club_sub.columns.values:
        for i in Lending_club_sub.columns.values:
            #print(i)
            if Lending_club_sub[i].dtypes == object or Lending_club_sub[i].dtypes == 'datetime64[ns]':
                Lending_club_sub_group = Lending_club_sub.groupby(i).size()
                maj = Lending_club_sub_group[Lending_club_sub_group == Lending_club_sub_group.max()].index[0]
                Lending_club_sub[i].fillna(maj, inplace = True)
            else:
                Lending_club_sub[i].fillna(Lending_club_sub[i].mean(skipna = True), inplace = True);
    return Lending_club_sub
Lending_club_sub = Clean_sub_Lending_club(Lending_club)    
    

pick out only the 3 year terms and the loans that are either finished or charged off

In [ ]:
Lending_club_sub['loan_status1'] = Lending_club['loan_status']
Lending_Club_3y = Lending_club_sub[Lending_club_sub['term'] == " 36 months" ]
Lending_Club_3yago = Lending_Club_3y[Lending_Club_3y['issue_d']<Lending_Club_3y['issue_d'].max() - timedelta(days = 365*3)]

In [ ]:
Lending_Club_3yago.loc[Lending_Club_3yago['loan_status1'] == 'Fully Paid','loan_status1'] = 0
Lending_Club_3yago.loc[Lending_Club_3yago['loan_status1'] != 0,'loan_status1'] = 1

In [ ]:
Lending_Club_3yago['duration'] = Lending_Club_3yago['duration'].apply(lambda x:x.days)
Lending_Club_3yago[['loan_status1','duration','grade']].dropna(inplace = True)

The KaplanMeierFitter fit a survival curve.

In [ ]:
kmf.fit(Lending_Club_3yago['duration'], event_observed = Lending_Club_3yago['loan_status1'])
kmf.survival_function_
kmf.median_
kmf.plot()

In [ ]:
GA =(Lending_Club_3yago['grade'] =='A')
GB =(Lending_Club_3yago['grade'] =='B')
GC =(Lending_Club_3yago['grade'] =='C')
GD =(Lending_Club_3yago['grade'] =='D')
GE =(Lending_Club_3yago['grade'] =='E')  
GF =(Lending_Club_3yago['grade'] =='F')  
GG =(Lending_Club_3yago['grade'] =='G') 

fit one curve by each grade

In [ ]:
ax = plt.subplot(111)

kmf.fit(Lending_Club_3yago['duration'][GA], event_observed=Lending_Club_3yago['loan_status1'][GA],label='A')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GB], event_observed=Lending_Club_3yago['loan_status1'][GB],label='B')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GC], event_observed=Lending_Club_3yago['loan_status1'][GC],label='C')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GD], event_observed=Lending_Club_3yago['loan_status1'][GD],label='D')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GE], event_observed=Lending_Club_3yago['loan_status1'][GE],label='E')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GF], event_observed=Lending_Club_3yago['loan_status1'][GF],label='F')
kmf.plot(ax= ax)
kmf.fit(Lending_Club_3yago['duration'][GG], event_observed=Lending_Club_3yago['loan_status1'][GG],label='G')
kmf.plot(ax= ax)
plt.xlim(0, 1100)




line = plt.gca().get_lines()[0]
agrade = zip(line.get_xdata(),line.get_ydata())
xyda = [x for x in agrade if x[0] < 1000]# and x[1] < 0.999]
xda = [x[0] for x in xyda]
yda = [x[1] for x in xyda]

line = plt.gca().get_lines()[1]
bgrade = zip(line.get_xdata(),line.get_ydata())
xydb = [x for x in bgrade if x[0] < 1000]# and x[1] < 0.999]
xdb = [x[0] for x in xydb]
ydb = [x[1] for x in xydb]

line = plt.gca().get_lines()[2]
cgrade = zip(line.get_xdata(),line.get_ydata())
xydc = [x for x in cgrade if x[0] < 1000]# and x[1] < 0.999]
xdc = [x[0] for x in xydc]
ydc = [x[1] for x in xydc]

line = plt.gca().get_lines()[3]
dgrade = zip(line.get_xdata(),line.get_ydata())
xydd = [x for x in dgrade if x[0] < 1000]# and x[1] < 0.999]
xdd = [x[0] for x in xydd]
ydd = [x[1] for x in xydd]

line = plt.gca().get_lines()[4]
egrade = zip(line.get_xdata(),line.get_ydata())
xyde = [x for x in egrade if x[0] < 1000]# and x[1] < 0.999]
xde = [x[0] for x in xyde]
yde = [x[1] for x in xyde]


line = plt.gca().get_lines()[5]
fgrade = zip(line.get_xdata(),line.get_ydata())
xydf = [x for x in fgrade if x[0] < 1000 ]#and x[1] < 0.999]
xdf = [x[0] for x in xydf]
ydf = [x[1] for x in xydf]

line = plt.gca().get_lines()[5]
ggrade = zip(line.get_xdata(),line.get_ydata())
xydg = [x for x in ggrade if x[0] < 1000 ]#and x[1] < 0.999]
xdg = [x[0] for x in xydg]
ydg = [x[1] for x in xydg]

fit it with a exponential decay line

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x + c)
def fit_plot(xda,yda):
    xda, yda = np.asarray(xda),np.asarray(yda) 
    
    popta, pcova = sp.optimize.curve_fit(func, xda, yda, p0 = (6.01891403e-01,   3.14864032e-04,   5.57748279e-01))
    a,b,c = popta
    plt.plot(xda, func(xda, *popta), 'r-',label="y = {0:.3}exp(-{1:.3}x + {2:.3}) ".format(a,b,c))
    plt.plot(xda,yda,label="Loan Survival")
#    plt.xlim(200,1100)
#    plt.ylim(0,1)
    plt.legend()
    print(func(1000, *popta) )
    return popta

In [ ]:
param_a = fit_plot(xda,yda)
param_b = fit_plot(xdb,ydb)
param_c = fit_plot(xdc,ydc)
param_d = fit_plot(xdd,ydd)
param_e = fit_plot(xde,yde)
param_f = fit_plot(xdf,ydf)
param_g = fit_plot(xdg,ydg)